In [1]:
from tkinter import *
from tkinter import messagebox
from experta import *
from PIL import Image, ImageTk, ImageOps
from PIL.Image import LANCZOS

class Movie(Fact):
    pass

class UserPreference(Fact):
    pass

class Recommendations(Fact):
    pass

class MovieRecommender(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.recommendations = []
        self.preferences = {}

    def validate_input(self, prompt, data_type):
        while True:
            try:
                value = data_type(prompt)
                return value
            except ValueError:
                messagebox.showinfo("Invalid Input", f"Invalid input. Please enter a valid {data_type.__name__}.")

    @Rule(
        AS.pref << UserPreference(
            genre=MATCH.genre, cf=MATCH.cf, rating=MATCH.rating,
            language=MATCH.pref_language, release_year_range=MATCH.release_year_range, runtime=MATCH.runtime,
            maxruntime=MATCH.maxruntime
        ),
        AS.movie << Movie(
            genre=MATCH.genre, title=MATCH.title, rating=MATCH.m_rating, cf=MATCH.m_cf,
            language=MATCH.language, release_year=MATCH.release_year, runtime=MATCH.m_runtime, img=MATCH.img
        ),
        TEST(lambda cf, m_cf: cf * m_cf > 0.6),
        TEST(lambda rating, m_rating: rating <= m_rating),
        TEST(lambda pref_language, language: pref_language.lower() == language.lower()),
        TEST(lambda release_year_range, release_year: release_year in release_year_range),
        TEST(lambda runtime, m_runtime: runtime <= m_runtime),
        TEST(lambda maxruntime, m_runtime: maxruntime >= m_runtime),
    )
    def recommend_movie(self, pref, movie, title, img):
        cf = 0.9 * min(pref["cf"], movie["cf"])
        self.declare(Recommendations(title=title, cf=cf, img=img))
        self.recommendations.append((title, cf, img))

    @Rule(
        AS.f1 << Recommendations(title=MATCH.title, cf=MATCH.cf1),
        AS.f2 << Recommendations(title=MATCH.title, cf=MATCH.cf2),
        TEST(lambda f1, f2: f1 != f2),
    )
    def combine(self, f1, f2, cf1, cf2):
        self.retract(f1)
        cf = cf1 + cf2 * (1 - cf1)
        self.modify(f2, cf=cf)

    def ask_preferences(self):
        root = Tk()
        root.title("Movie Recommender")

        def submit_preferences():
            self.preferences["genre"] = genre_entry.get()
            self.preferences["cf"] = float(cf_entry.get())
            self.preferences["rating"] = float(rating_entry.get())
            self.preferences["language"] = language_entry.get()
            start_year = int(start_year_entry.get())
            end_year = int(end_year_entry.get())
            self.preferences["release_year_range"] = list(range(start_year, end_year + 1))
            self.preferences["runtime"] = int(runtime_entry.get())
            self.preferences["maxruntime"] = int(maxruntime_entry.get())
            root.destroy()

        # Create GUI elements
        label_font = ("Arial", 12, "bold")
        entry_font = ("Arial", 12)

        # Genre
        genre_label = Label(root, text="Preferred Genre:", font=label_font)
        genre_label.pack()
        genre_entry = Entry(root, font=entry_font)
        genre_entry.pack()

        # CF
        cf_label = Label(root, text="Certainty Factor (0-1):", font=label_font)
        cf_label.pack()
        cf_entry = Entry(root, font=entry_font)
        cf_entry.pack()

        # Rating
        rating_label = Label(root, text="Minimum Rating (0-10):", font=label_font)
        rating_label.pack()
        rating_entry = Entry(root, font=entry_font)
        rating_entry.pack()

        # Language
        language_label = Label(root, text="Preferred Language:", font=label_font)
        language_label.pack()
        language_entry = Entry(root, font=entry_font)
        language_entry.pack()

        # Release Year Range
        start_year_label = Label(root, text="Start Year:", font=label_font)
        start_year_label.pack()
        start_year_entry = Entry(root, font=entry_font)
        start_year_entry.pack()

        end_year_label = Label(root, text="End Year:", font=label_font)
        end_year_label.pack()
        end_year_entry = Entry(root, font=entry_font)
        end_year_entry.pack()

        # Runtime
        runtime_label = Label(root, text="Minimum Runtime:", font=label_font)
        runtime_label.pack()
        runtime_entry = Entry(root, font=entry_font)
        runtime_entry.pack()

        # Max Runtime
        maxruntime_label = Label(root, text="Maximum Runtime:", font=label_font)
        maxruntime_label.pack()
        maxruntime_entry = Entry(root, font=entry_font)
        maxruntime_entry.pack()

        # Submit Button
        submit_button = Button(root, text="Submit", font=label_font, command=submit_preferences)
        submit_button.pack()

        root.mainloop()

        return self.preferences

    def display_recommendations(self, recommendations):
        root = Tk()
        root.title("Recommended Movies")

        # Create a scrollable frame
        canvas = Canvas(root)
        canvas.pack(side=LEFT, fill=BOTH, expand=True)

        scrollbar = Scrollbar(root, command=canvas.yview)
        scrollbar.pack(side=RIGHT, fill=Y)

        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

        frame = Frame(canvas)
        canvas.create_window((0, 0), window=frame, anchor=NW)

        for recommendation in recommendations:
            rec_frame = Frame(frame, padx=10, pady=10)
            rec_frame.pack()

            image = Image.open(recommendation[2])
            image = image.resize((200, 300), Image.LANCZOS)
            photo = ImageTk.PhotoImage(image)

            label = Label(rec_frame, image=photo)
            label.pack(side=LEFT)

            title_label = Label(rec_frame, text="Recommended Movie: " + recommendation[0])
            title_label.pack()

            cf_label = Label(rec_frame, text="Certainty Factor: " + str(recommendation[1]))
            cf_label.pack()

            # Keep a reference to the photo to prevent garbage collection
            label.photo = photo

        root.mainloop()

    def reset_recommendations(self):
        self.recommendations = []

    def get_recommendations(self):
        return self.recommendations

def main():
    recommender = MovieRecommender()
    movies = [
    {"title": "The Shawshank Redemption", "genre": "Drama", "rating": 9.3, "cf": 0.95, "language": "English", "release_year": 1994, "runtime": 142, "img": "DataBase/The Shawshank Redemption.jfif"},
    {"title": "The Godfather", "genre": "Crime", "rating": 9.2, "cf": 0.90, "language": "English", "release_year": 1972, "runtime": 175, "img": "DataBase/The Godfather.jfif"},
    {"title": "Pulp Fiction", "genre": "Crime", "rating": 8.9, "cf": 0.85, "language": "English", "release_year": 1994, "runtime": 154, "img": "DataBase/Pulp Fiction.jfif"},
    {"title": "The Dark Knight", "genre": "Action", "rating": 9.0, "cf": 0.95, "language": "English", "release_year": 2008, "runtime": 152, "img": "DataBase/The Dark Knight.jfif"},
    {"title": "Inception", "genre": "Sci-Fi", "rating": 8.8, "cf": 0.90, "language": "English", "release_year": 2010, "runtime": 148, "img": "DataBase/Inception.jfif"},
    {"title": "Interstellar", "genre": "Sci-Fi", "rating": 8.6, "cf": 0.85, "language": "English", "release_year": 2014, "runtime": 169, "img": "DataBase/Interstellar.jfif"},
    {"title": "The Godfather: Part II", "genre": "Crime", "rating": 9.0, "cf": 0.92, "language": "English", "release_year": 1974, "runtime": 202, "img": "DataBase/The Godfather Part II.jfif"},
    {"title": "Schindler's List", "genre": "Drama", "rating": 8.9, "cf": 0.90, "language": "English", "release_year": 1993, "runtime": 195, "img": "DataBase/Schindler's List.jfif"},
    {"title": "Forrest Gump", "genre": "Drama", "rating": 8.8, "cf": 0.85, "language": "English", "release_year": 1994, "runtime": 142, "img": "DataBase/Forrest Gump.jfif"},
    {"title": "The Silence of the Lambs", "genre": "Crime", "rating": 8.6, "cf": 0.88, "language": "English", "release_year": 1991, "runtime": 118, "img": "DataBase/The Silence of the Lambs.jfif"},
    {"title": "Saving Private Ryan", "genre": "Drama", "rating": 8.6, "cf": 0.86, "language": "English", "release_year": 1998, "runtime": 169, "img": "DataBase/Saving Private Ryan.jfif"},
    {"title": "The Matrix", "genre": "Sci-Fi", "rating": 8.7, "cf": 0.87, "language": "English", "release_year": 1999, "runtime": 136, "img": "DataBase/The Matrix.jfif"},
    {"title": "The Green Mile", "genre": "Drama", "rating": 8.6, "cf": 0.89, "language": "English", "release_year": 1999, "runtime": 189, "img": "DataBase/The Green Mile.jfif"},
    {"title": "Fight Club", "genre": "Drama", "rating": 8.8, "cf": 0.85, "language": "English", "release_year": 1999, "runtime": 139, "img": "DataBase/Fight Club.jfif"},
    {"title": "The Prestige", "genre": "Drama", "rating": 8.5, "cf": 0.92, "language": "English", "release_year": 2006, "runtime": 130, "img": "DataBase/The Prestige.jfif"},
    {"title": "The Departed", "genre": "Crime", "rating": 8.5, "cf": 0.88, "language": "English", "release_year": 2006, "runtime": 151, "img": "DataBase/The Departed.jfif"},
    {"title": "The Revenant", "genre": "Drama", "rating": 8.0, "cf": 0.91, "language": "English", "release_year": 2015, "runtime": 156, "img": "DataBase/The Revenant.jfif"},
    {"title": "Dunkirk", "genre": "Drama", "rating": 7.9, "cf": 0.86, "language": "English", "release_year": 2017, "runtime": 106, "img": "DataBase/Dunkirk.jfif"},
    {"title": "The Lord of the Rings: The Fellowship of the Ring", "genre": "Fantasy", "rating": 8.8, "cf": 0.89, "language": "English", "release_year": 2001, "runtime": 178, "img": "DataBase/The Lord of the Rings The Fellowship of the Ring.jfif"},
    {"title": "The Lord of the Rings: The Two Towers", "genre": "Fantasy", "rating": 8.7, "cf": 0.88, "language": "English", "release_year": 2002, "runtime": 179, "img": "DataBase/The Lord of the Rings The Two Towers.jfif"},
    {"title": "The Lord of the Rings: The Return of the King", "genre": "Fantasy", "rating": 8.9, "cf": 0.90, "language": "English", "release_year": 2003, "runtime": 201, "img": "DataBase/The Lord of the Rings The Return of the King.jfif"},
    {"title": "The Avengers", "genre": "Action", "rating": 8.0, "cf": 0.85, "language": "English", "release_year": 2012, "runtime": 143, "img": "DataBase/The Avengers.jfif"},
    {"title": "La La Land", "genre": "Musical", "rating": 8.0, "cf": 0.85, "language": "English", "release_year": 2016, "runtime": 128, "img": "DataBase/La La Land.jfif"},
    {"title": "The Social Network", "genre": "Drama", "rating": 7.7, "cf": 0.82, "language": "English", "release_year": 2010, "runtime": 120, "img": "DataBase/The Social Network.jfif"},
    {"title": "Her", "genre": "Romance", "rating": 8.0, "cf": 0.85, "language": "English", "release_year": 2013, "runtime": 126, "img": "DataBase/Her.jfif"},
    {"title": "Parasite", "genre": "Drama", "rating": 8.6, "cf": 0.89, "language": "Korean", "release_year": 2019, "runtime": 132, "img": "DataBase/Parasite.jfif"},
    {"title": "Joker", "genre": "Drama", "rating": 8.4, "cf": 0.87, "language": "English", "release_year": 2019, "runtime": 122, "img": "DataBase/Joker.jfif"},
    {"title": "The Godfather: Part III", "genre": "Crime", "rating": 7.6, "cf": 0.82, "language": "English", "release_year": 1990, "runtime": 162, "img": "DataBase/The Godfather Part III.jfif"},
    {"title": "Goodfellas", "genre": "Crime", "rating": 8.7, "cf": 0.91, "language": "English", "release_year": 1990, "runtime": 146, "img": "DataBase/Goodfellas.jfif"},
    {"title": "The Silence of the Lambs", "genre": "Crime", "rating": 8.6, "cf": 0.88, "language": "English", "release_year": 1991, "runtime": 118, "img": "DataBase/The Silence of the Lambs.jfif"},
    {"title": "Jurassic Park", "genre": "Sci-Fi", "rating": 8.1, "cf": 0.84, "language": "English", "release_year": 1993, "runtime": 127, "img": "DataBase/Jurassic Park.jfif"},
    {"title": "The Lion King", "genre": "Animation", "rating": 8.5, "cf": 0.87, "language": "English", "release_year": 1994, "runtime": 88, "img": "DataBase/The Lion King.jfif"},
    {"title": "The Usual Suspects", "genre": "Crime", "rating": 8.5, "cf": 0.86, "language": "English", "release_year": 1995, "runtime": 106, "img": "DataBase/The Usual Suspects.jfif"},
    {"title": "Fargo", "genre": "Crime", "rating": 8.1, "cf": 0.83, "language": "English", "release_year": 1996, "runtime": 98, "img": "DataBase/Fargo.jfif"},
    {"title": "The Sixth Sense", "genre": "Thriller", "rating": 8.1, "cf": 0.85, "language": "English", "release_year": 1999, "runtime": 107, "img": "DataBase/The Sixth Sense.jfif"},
    {"title": "Memento", "genre": "Thriller", "rating": 8.4, "cf": 0.89, "language": "English", "release_year": 2000, "runtime": 113, "img": "DataBase/Memento.jfif"},
    {"title": "City of God", "genre": "Drama", "rating": 8.6, "cf": 0.88, "language": "Portuguese", "release_year": 2002, "runtime": 130, "img": "DataBase/City of God.jfif"},
    {"title": "Eternal Sunshine of the Spotless Mind", "genre": "Romance", "rating": 8.3, "cf": 0.87, "language": "English", "release_year": 2004, "runtime": 108, "img": "DataBase/Eternal Sunshine of the Spotless Mind.jfif"},
    {"title": "The Prestige", "genre": "Drama", "rating": 8.5, "cf": 0.92, "language": "English", "release_year": 2006, "runtime": 130, "img": "DataBase/The Prestige.jfif"},
    {"title": "No Country for Old Men", "genre": "Crime", "rating": 8.1, "cf": 0.86, "language": "English", "release_year": 2007, "runtime": 122, "img": "DataBase/No Country for Old Men.jfif"},
    {"title": "Avatar", "genre": "Sci-Fi", "rating": 7.8, "cf": 0.83, "language": "English", "release_year": 2009, "runtime": 162, "img": "DataBase/Avatar.jfif"},
    {"title": "Black Swan", "genre": "Thriller", "rating": 8.0, "cf": 0.86, "language": "English", "release_year": 2010, "runtime": 108, "img": "DataBase/Black Swan.jfif"},
    {"title": "The Girl with the Dragon Tattoo", "genre": "Thriller", "rating": 7.8, "cf": 0.84, "language": "English", "release_year": 2011, "runtime": 158, "img": "DataBase/The Girl with the Dragon Tattoo.jfif"},
    {"title": "The Imitation Game", "genre": "Drama", "rating": 8.0, "cf": 0.87, "language": "English", "release_year": 2014, "runtime": 114, "img": "DataBase/The Imitation Game.jfif"},
    {"title": "Mad Max: Fury Road", "genre": "Action", "rating": 8.1, "cf": 0.86, "language": "English", "release_year": 2015, "runtime": 120, "img": "DataBase/Mad Max Fury Road.jfif"},
    {"title": "Moonlight", "genre": "Drama", "rating": 7.4, "cf": 0.81, "language": "English", "release_year": 2016, "runtime": 111, "img": "DataBase/Moonlight.jfif"},
    {"title": "Get Out", "genre": "Horror", "rating": 7.7, "cf": 0.83, "language": "English", "release_year": 2017, "runtime": 104, "img": "DataBase/Get Out.jfif"},
    {"title": "Blade Runner 2049", "genre": "Sci-Fi", "rating": 8.0, "cf": 0.87, "language": "English", "release_year": 2017, "runtime": 164, "img": "DataBase/Blade Runner 2049.jfif"},
    {"title": "Three Billboards Outside Ebbing, Missouri", "genre": "Drama", "rating": 8.2, "cf": 0.89, "language": "English", "release_year": 2017, "runtime": 115, "img": "DataBase/Three Billboards Outside Ebbing Missouri.jfif"},
    {"title": "The Shape of Water", "genre": "Fantasy", "rating": 7.3, "cf": 0.81, "language": "English", "release_year": 2017, "runtime": 123, "img": "DataBase/The Shape of Water.jfif"},
    {"title": "A Star is Born", "genre": "Drama", "rating": 7.7, "cf": 0.83, "language": "English", "release_year": 2018, "runtime": 136, "img": "DataBase/A Star is Born.jfif"},
    {"title": "Black Panther", "genre": "Action", "rating": 7.3, "cf": 0.81, "language": "English", "release_year": 2018, "runtime": 134, "img": "DataBase/Black Panther.jfif"},
    {"title": "Bohemian Rhapsody", "genre": "Biography", "rating": 8.0, "cf": 0.88, "language": "English", "release_year": 2018, "runtime": 134, "img": "DataBase/Bohemian Rhapsody.jfif"},
    {"title": "Green Book", "genre": "Drama", "rating": 8.2, "cf": 0.89, "language": "English", "release_year": 2018, "runtime": 130, "img": "DataBase/Green Book.jfif"},
    {"title": "1917", "genre": "War", "rating": 8.3, "cf": 0.88, "language": "English", "release_year": 2019, "runtime": 119, "img": "DataBase/1917.jfif"},
    {"title": "The Irishman", "genre": "Crime", "rating": 7.9, "cf": 0.84, "language": "English", "release_year": 2019, "runtime": 209, "img": "DataBase/The Irishman.jfif"},
    {"title": "The Gentlemen", "genre": "Action", "rating": 7.8, "cf": 0.83, "language": "English", "release_year": 2019, "runtime": 113, "img": "DataBase/The Gentlemen.jfif"},
    {"title": "Soul", "genre": "Animation", "rating": 8.1, "cf": 0.85, "language": "English", "release_year": 2020, "runtime": 100, "img": "DataBase/Soul.jfif"},
    {"title": "Nomadland", "genre": "Drama", "rating": 7.3, "cf": 0.81, "language": "English", "release_year": 2020, "runtime": 107, "img": "DataBase/Nomadland.jfif"},
    {"title": "The Trial of the Chicago 7", "genre": "Drama", "rating": 7.8, "cf": 0.83, "language": "English", "release_year": 2020, "runtime": 129, "img": "DataBase/The Trial of the Chicago 7.jfif"},
    {"title": "Sound of Metal", "genre": "Drama", "rating": 7.8, "cf": 0.83, "language": "English", "release_year": 2020, "runtime": 120, "img": "DataBase/Sound of Metal.jfif"},
    {"title": "Tenet", "genre": "Sci-Fi", "rating": 7.5, "cf": 0.82, "language": "English", "release_year": 2020, "runtime": 150, "img": "DataBase/Tenet.jfif"},
    {"title": "The Father", "genre": "Drama", "rating": 8.3, "cf": 0.88, "language": "English", "release_year": 2020, "runtime": 97, "img": "DataBase/The Father.jfif"},
    {"title": "Gone with the Wind", "genre": "Drama", "rating": 8.1, "release_year": 1939, "runtime": 238, "language": "English", "img": "DataBase/Gone with the Wind.jfif"},
    {"title": "Casablanca", "genre": "Drama", "rating": 8.5, "release_year": 1942, "runtime": 102, "language": "English", "img": "DataBase/Casablanca.jfif"},
    {"title": "Citizen Kane", "genre": "Drama", "rating": 8.3, "release_year": 1941, "runtime": 119, "language": "English", "img": "DataBase/Citizen Kane.jfif"},
    {"title": "The Wizard of Oz", "genre": "Adventure", "rating": 8.0, "release_year": 1939, "runtime": 102, "language": "English", "img": "DataBase/The Wizard of Oz.jfif"},
    {"title": "Lawrence of Arabia", "genre": "Biography", "rating": 8.3, "release_year": 1962, "runtime": 228, "language": "English", "img": "DataBase/Lawrence of Arabia.jfif"},
    {"title": "Star Wars: Episode IV - A New Hope", "genre": "Action", "rating": 8.6, "release_year": 1977, "runtime": 121, "language": "English", "img": "DataBase/Star Wars Episode IV - A New Hope.jfif"},
    {"title": "The Great Gatsby", "genre": "Drama", "rating": 7.2, "release_year": 2013, "runtime": 143, "language": "English", "img": "DataBase/The Great Gatsby.jfif"},
    {"title": "The Grand Budapest Hotel", "genre": "Adventure", "rating": 8.1, "release_year": 2014, "runtime": 99, "language": "English", "img": "DataBase/The Grand Budapest Hotel.jfif"}
    ]

    recommender.reset()
    for movie in movies:
        recommender.declare(Movie(**movie))

    preferences = recommender.ask_preferences()
    recommender.declare(UserPreference(**preferences))
    recommender.run()

    recommendations = recommender.get_recommendations()
    recommendations.sort(key=lambda rec: rec[1], reverse=True)  # sorted in descending order based on the certainty factor

    if recommendations:
        print("Recommended Movies:")
        for recommendation in recommendations:
            print("Title:", recommendation[0])
            print("Certainty Factor:", recommendation[1])
            print()

        recommender.display_recommendations(recommendations)
    else:
        messagebox.showinfo("No Movie Recommendations", "No movie recommendations.")

if __name__ == "__main__":
    main()


C:\Users\mudar\AppData\Local\Temp\ipykernel_3544\262363786.py:5: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  from PIL.Image import LANCZOS


Recommended Movies:
Title: The Shawshank Redemption
Certainty Factor: 0.855

Title: The Prestige
Certainty Factor: 0.8280000000000001

Title: Schindler's List
Certainty Factor: 0.81

Title: The Green Mile
Certainty Factor: 0.801

Title: Saving Private Ryan
Certainty Factor: 0.774

Title: Fight Club
Certainty Factor: 0.765

Title: Forrest Gump
Certainty Factor: 0.765

Title: The Social Network
Certainty Factor: 0.738



C:\Users\mudar\AppData\Local\Temp\ipykernel_3544\262363786.py:159: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((200, 300), Image.LANCZOS)


In [1]:
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk
from experta import *

class Movie(Fact):
    pass

class UserPreference(Fact):
    pass

class Recommendations(Fact):
    pass

class MovieRecommender(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.recommendations = []
        self.preferences = {}

    def validate_input(self, prompt, data_type):
        while True:
            try:
                value = data_type(prompt)
                return value
            except ValueError:
                messagebox.showinfo("Invalid Input", f"Invalid input. Please enter a valid {data_type.__name__}.")

    @Rule(
        AS.pref << UserPreference(
            genre=MATCH.genre, cf=MATCH.cf, rating=MATCH.rating,
            language=MATCH.pref_language, release_year_range=MATCH.release_year_range, runtime=MATCH.runtime,
            maxruntime=MATCH.maxruntime
        ),
        AS.movie << Movie(
            genre=MATCH.genre, title=MATCH.title, rating=MATCH.m_rating, cf=MATCH.m_cf,
            language=MATCH.language, release_year=MATCH.release_year, runtime=MATCH.m_runtime, img=MATCH.img
        ),
        TEST(lambda cf, m_cf: cf * m_cf > 0.6),
        TEST(lambda rating, m_rating: rating <= m_rating),
        TEST(lambda pref_language, language: pref_language.lower() == language.lower()),
        TEST(lambda release_year_range, release_year: release_year in release_year_range),
        TEST(lambda runtime, m_runtime: runtime <= m_runtime),
        TEST(lambda maxruntime, m_runtime: maxruntime >= m_runtime),
    )
    def recommend_movie(self, pref, movie, title, img):
        self.declare(Recommendations(title=title, cf=0.9 * min(pref["cf"], movie["cf"]), img=img))
        self.recommendations.append((title, pref["cf"] * movie["cf"], img))

    @Rule(
        AS.f1 << Recommendations(title=MATCH.title, cf=MATCH.cf1),
        AS.f2 << Recommendations(title=MATCH.title, cf=MATCH.cf2),
        TEST(lambda f1, f2: f1 != f2),
    )
    def combine(self, f1, f2, cf1, cf2):
        self.retract(f1)
        cf = cf1 + cf2 * (1 - cf1)
        self.modify(f2, cf=cf)

    def ask_preferences(self):
        root = Tk()
        root.title("Movie Recommender")

        genre_label = Label(root, text="Preferred Genre:")
        genre_label.pack()
        genre_entry = Entry(root)
        genre_entry.pack()

        cf_label = Label(root, text="Certainty Factor (0-1):")
        cf_label.pack()
        cf_entry = Entry(root)
        cf_entry.pack()

        rating_label = Label(root, text="Minimum Rating (0-10):")
        rating_label.pack()
        rating_entry = Entry(root)
        rating_entry.pack()

        language_label = Label(root, text="Preferred Language:")
        language_label.pack()
        language_entry = Entry(root)
        language_entry.pack()

        start_year_label = Label(root, text="Start Year:")
        start_year_label.pack()
        start_year_entry = Entry(root)
        start_year_entry.pack()

        end_year_label = Label(root, text="End Year:")
        end_year_label.pack()
        end_year_entry = Entry(root)
        end_year_entry.pack()

        runtime_label = Label(root, text="Minimum Runtime:")
        runtime_label.pack()
        runtime_entry = Entry(root)
        runtime_entry.pack()

        maxruntime_label = Label(root, text="Maximum Runtime:")
        maxruntime_label.pack()
        maxruntime_entry = Entry(root)
        maxruntime_entry.pack()

        def submit_preferences():
            self.preferences["genre"] = genre_entry.get()
            self.preferences["cf"] = float(cf_entry.get())
            self.preferences["rating"] = float(rating_entry.get())
            self.preferences["language"] = language_entry.get()
            start_year = int(start_year_entry.get())
            end_year = int(end_year_entry.get())
            self.preferences["release_year_range"] = list(range(start_year, end_year + 1))
            self.preferences["runtime"] = int(runtime_entry.get())
            self.preferences["maxruntime"] = int(maxruntime_entry.get())
            root.destroy()

        submit_button = Button(root, text="Submit", command=submit_preferences)
        submit_button.pack()

        root.mainloop()

        return self.preferences

    def display_recommendation(self, recommendation):
        root = Tk()
        root.title("Recommended Movie")

        image = Image.open(recommendation[2])
        image = image.resize((200, 300), Image.ANTIALIAS)
        photo = ImageTk.PhotoImage(image)

        label = Label(root, image=photo)
        label.pack()

        title_label = Label(root, text="Recommended Movie: " + recommendation[0])
        title_label.pack()

        cf_label = Label(root, text="Certainty Factor: " + str(recommendation[1]))
        cf_label.pack()

        root.mainloop()

    def reset_recommendations(self):
        self.recommendations = []

    def get_recommendations(self):
        return self.recommendations

def main():
    recommender = MovieRecommender()
    movies = [
    {"title": "The Shawshank Redemption", "genre": "Drama", "rating": 9.3, "cf": 0.95, "language": "English", "release_year": 1994, "runtime": 142, "img": "DataBase/The Shawshank Redemption.jfif"},
    
    ]

    recommender.reset()
    for movie in movies:
        recommender.declare(Movie(**movie))

    preferences = recommender.ask_preferences()
    recommender.declare(UserPreference(**preferences))
    recommender.run()

    recommendations = recommender.get_recommendations()
    recommendations.sort(key=lambda rec: rec[1], reverse=True)

    if recommendations:
        top_recommendation = recommendations[0]
        certainty_factor = round(2 * top_recommendation[1])
        print("Recommended Movie:", top_recommendation[0])
        print("Certainty Factor:", certainty_factor)

        root = Tk()
        root.title("Recommended Movie")

        image = Image.open(top_recommendation[2])
        image = image.resize((200, 300), Image.ANTIALIAS)
        photo = ImageTk.PhotoImage(image)

        label = Label(root, image=photo)
        label.pack()

        title_label = Label(root, text="Recommended Movie: " + top_recommendation[0])
        title_label.pack()

        cf_label = Label(root, text="Certainty Factor: " + str(top_recommendation[1]))
        cf_label.pack()

        root.mainloop()
    else:
        root = Tk()
        root.title("No Movie Recommendations")

        label = Label(root, text="No movie recommendations.")
        label.pack()

        root.mainloop()

if __name__ == "__main__":
    main()


Recommended Movie: The Shawshank Redemption
Certainty Factor: 2


C:\Users\mudar\AppData\Local\Temp\ipykernel_19088\1747702409.py:176: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((200, 300), Image.ANTIALIAS)


In [1]:
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QGridLayout, QLineEdit, QMessageBox
from PyQt5.QtGui import QPixmap, QImage
from PIL import Image
from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap
from PIL import Image
from experta import *

class Movie(Fact):
    pass

class UserPreference(Fact):
    pass

class Recommendations(Fact):
    pass

class MovieRecommender(KnowledgeEngine):
    def __init__(self):
        super().__init__()
        self.recommendations = []
        self.preferences = {}

    def validate_input(self, prompt, data_type):
        while True:
            try:
                value = data_type(prompt)
                return value
            except ValueError:
                QMessageBox.information(None, "Invalid Input", f"Invalid input. Please enter a valid {data_type.__name__}.")

    @Rule(
        AS.pref << UserPreference(
            genre=MATCH.genre, cf=MATCH.cf, rating=MATCH.rating,
            language=MATCH.pref_language, release_year_range=MATCH.release_year_range, runtime=MATCH.runtime,
            maxruntime=MATCH.maxruntime
        ),
        AS.movie << Movie(
            genre=MATCH.genre, title=MATCH.title, rating=MATCH.m_rating, cf=MATCH.m_cf,
            language=MATCH.language, release_year=MATCH.release_year, runtime=MATCH.m_runtime, img=MATCH.img
        ),
        TEST(lambda cf, m_cf: cf * m_cf > 0.6),
        TEST(lambda rating, m_rating: rating <= m_rating),
        TEST(lambda pref_language, language: pref_language.lower() == language.lower()),
        TEST(lambda release_year_range, release_year: release_year in release_year_range),
        TEST(lambda runtime, m_runtime: runtime <= m_runtime),
        TEST(lambda maxruntime, m_runtime: maxruntime >= m_runtime),
    )
    def recommend_movie(self, pref, movie, title, img):
        self.declare(Recommendations(title=title, cf=0.9 * min(pref["cf"], movie["cf"]), img=img))
        self.recommendations.append((title, pref["cf"] * movie["cf"], img))

    @Rule(
        AS.f1 << Recommendations(title=MATCH.title, cf=MATCH.cf1),
        AS.f2 << Recommendations(title=MATCH.title, cf=MATCH.cf2),
        TEST(lambda f1, f2: f1 != f2),
    )
    def combine(self, f1, f2, cf1, cf2):
        self.retract(f1)
        cf = cf1 + cf2 * (1 - cf1)
        self.modify(f2, cf=cf)

    def ask_preferences(self):
        app = QApplication([])
        window = QWidget()
        window.setWindowTitle("Movie Recommender")

        grid_layout = QGridLayout()

        genre_label = QLabel("Preferred Genre:")
        genre_entry = QLineEdit()
        grid_layout.addWidget(genre_label, 0, 0)
        grid_layout.addWidget(genre_entry, 0, 1)

        cf_label = QLabel("Certainty Factor (0-1):")
        cf_entry = QLineEdit()
        grid_layout.addWidget(cf_label, 1, 0)
        grid_layout.addWidget(cf_entry, 1, 1)

        rating_label = QLabel("Minimum Rating (0-10):")
        rating_entry = QLineEdit()
        grid_layout.addWidget(rating_label, 2, 0)
        grid_layout.addWidget(rating_entry, 2, 1)

        language_label = QLabel("Preferred Language:")
        language_entry = QLineEdit()
        grid_layout.addWidget(language_label, 3, 0)
        grid_layout.addWidget(language_entry, 3, 1)

        start_year_label = QLabel("Start Year:")
        start_year_entry = QLineEdit()
        grid_layout.addWidget(start_year_label, 4, 0)
        grid_layout.addWidget(start_year_entry, 4, 1)

        end_year_label = QLabel("End Year:")
        end_year_entry = QLineEdit()
        grid_layout.addWidget(end_year_label, 5, 0)
        grid_layout.addWidget(end_year_entry, 5, 1)

        runtime_label = QLabel("Minimum Runtime:")
        runtime_entry = QLineEdit()
        grid_layout.addWidget(runtime_label, 6, 0)
        grid_layout.addWidget(runtime_entry, 6, 1)

        maxruntime_label = QLabel("Maximum Runtime:")
        maxruntime_entry = QLineEdit()
        grid_layout.addWidget(maxruntime_label, 7, 0)
        grid_layout.addWidget(maxruntime_entry, 7, 1)

        submit_button = QPushButton("Submit")
        grid_layout.addWidget(submit_button, 8, 0, 1, 2)

        window.setLayout(grid_layout)

        def submit_preferences():
            self.preferences["genre"] = genre_entry.text()
            self.preferences["cf"] = float(cf_entry.text())
            self.preferences["rating"] = float(rating_entry.text())
            self.preferences["language"] = language_entry.text()
            start_year = int(start_year_entry.text())
            end_year = int(end_year_entry.text())
            self.preferences["release_year_range"] = list(range(start_year, end_year + 1))
            self.preferences["runtime"] = int(runtime_entry.text())
            self.preferences["maxruntime"] = int(maxruntime_entry.text())
            app.quit()

        submit_button.clicked.connect(submit_preferences)

        window.show()
        app.exec_()

        return self.preferences

    def display_recommendation(self, recommendation):
        app = QApplication([])
        window = QWidget()
        window.setWindowTitle("Recommended Movie")

        layout = QVBoxLayout()

        image_label = QLabel()
        image = Image.open(recommendation[2])
        image = image.resize((200, 300))
        q_image = QImage(image.tobytes(), image.width, image.height, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(q_image)
        image_label.setPixmap(pixmap)
        layout.addWidget(image_label)

        title_label = QLabel("Recommended Movie: " + recommendation[0])
        layout.addWidget(title_label)

        cf_label = QLabel("Certainty Factor: " + str(recommendation[1]))
        layout.addWidget(cf_label)

        window.setLayout(layout)
        window.show()

        app.exec_()

    def reset_recommendations(self):
        self.recommendations = []

    def get_recommendations(self):
        return self.recommendations

def main():
    recommender = MovieRecommender()
    movies = [
        {"title": "The Shawshank Redemption", "genre": "Drama", "rating": 9.3, "cf": 0.95, "language": "English", "release_year": 1994, "runtime": 142, "img": "DataBase/The Shawshank Redemption.jfif"},
    ]

    recommender.reset()
    for movie in movies:
        recommender.declare(Movie(**movie))

    preferences = recommender.ask_preferences()
    recommender.declare(UserPreference(**preferences))
    recommender.run()

    recommendations = recommender.get_recommendations()
    recommendations.sort(key=lambda rec: rec[1], reverse=True)

    if recommendations:
        top_recommendation = recommendations[0]
        certainty_factor = round(2 * top_recommendation[1])
        print("Recommended Movie:", top_recommendation[0])
        print("Certainty Factor:", certainty_factor)

        recommender.display_recommendation(top_recommendation)
    else:
        QMessageBox.information(None, "No Movie Recommendations", "No movie recommendations.")

if __name__ == "__main__":
    main()


ValueError: could not convert string to float: ''

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QLabel, QComboBox, QSlider, QPushButton, QTextEdit, QLineEdit, QSpinBox, QHBoxLayout
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import (
    QApplication,
    QMainWindow,
    QWidget,
    QVBoxLayout,
    QLabel,
    QComboBox,
    QSlider,
    QPushButton,
    QTextEdit,
    QSpinBox,
)
class MovieRecommenderApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Movie Recommender")
        self.setup_ui()

    def setup_ui(self):
        central_widget = QWidget()
        layout = QVBoxLayout()
        central_widget.setLayout(layout)

        title_label = QLabel("Movie Recommender")
        title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(title_label)

        genre_label = QLabel("Genre:")
        self.genre_combo = QComboBox()
        self.genre_combo.addItems(["", "Action", "Comedy", "Drama", "Horror", "Sci-Fi", "Thriller"])
        layout.addWidget(genre_label)
        layout.addWidget(self.genre_combo)

        # Add other preference widgets (cf, rating, language, release year, runtime)

        recommend_button = QPushButton("Recommend")
        recommend_button.clicked.connect(self.recommend_movies)
        layout.addWidget(recommend_button)

        self.result_textedit = QTextEdit()
        self.result_textedit.setReadOnly(True)
        layout.addWidget(self.result_textedit)

        self.setCentralWidget(central_widget)

    def recommend_movies(self):
        genre = self.genre_combo.currentText()
        # Get other preference values from widgets

        # Create MovieRecommender instance and recommend movies
        recommender = MovieRecommender()
        recommender.reset()
        recommender.declare(UserPreference(genre=genre))
        recommender.recommend()
        recommendations = recommender.recommendations

        if recommendations:
            result_text = "Recommended movies:\n"
            for rec in recommendations:
                result_text += f"- {rec[0]}\n"
        else:
            result_text = "No movies found matching your preferences."

        self.result_textedit.setPlainText(result_text)



# ... (Keep the rest of your existing code here)
class MovieRecommenderApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Movie Recommender")
        self.setup_ui()

    def setup_ui(self):
        central_widget = QWidget()
        layout = QVBoxLayout()
        central_widget.setLayout(layout)

        title_label = QLabel("Movie Recommender")
        title_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(title_label)

        genre_label = QLabel("Genre:")
        self.genre_combo = QComboBox()
        self.genre_combo.addItems(["", "Action", "Comedy", "Drama", "Horror", "Sci-Fi", "Thriller"])
        layout.addWidget(genre_label)
        layout.addWidget(self.genre_combo)

        cf_label = QLabel("Confidence Level:")
        self.cf_slider = QSlider(Qt.Orientation.Horizontal)
        self.cf_slider.setRange(0, 100)
        self.cf_slider.setValue(50)
        self.cf_slider.setTickPosition(QSlider.TicksBelow)
        self.cf_slider.setTickInterval(10)
        layout.addWidget(cf_label)
        layout.addWidget(self.cf_slider)

        rating_label = QLabel("Minimum Rating:")
        self.rating_slider = QSlider(Qt.Orientation.Horizontal)
        self.rating_slider.setRange(0, 100)
        self.rating_slider.setValue(70)
        self.rating_slider.setTickPosition(QSlider.TicksBelow)
        self.rating_slider.setTickInterval(10)
        layout.addWidget(rating_label)
        layout.addWidget(self.rating_slider)

        language_label = QLabel("Preferred Language:")
        self.language_edit = QLineEdit()
        layout.addWidget(language_label)
        layout.addWidget(self.language_edit)

        release_year_label = QLabel("Release Year Range:")
        self.release_year_layout = QHBoxLayout()
        self.release_year_spinbox_min = QSpinBox()
        self.release_year_spinbox_max = QSpinBox()
        self.release_year_layout.addWidget(self.release_year_spinbox_min)
        self.release_year_layout.addWidget(self.release_year_spinbox_max)
        layout.addWidget(release_year_label)
        layout.addLayout(self.release_year_layout)

        runtime_label = QLabel("Runtime Range (in minutes):")
        self.runtime_layout = QHBoxLayout()
        self.runtime_spinbox_min = QSpinBox()
        self.runtime_spinbox_max = QSpinBox()
        self.runtime_layout.addWidget(self.runtime_spinbox_min)
        self.runtime_layout.addWidget(self.runtime_spinbox_max)
        layout.addWidget(runtime_label)
        layout.addLayout(self.runtime_layout)

        recommend_button = QPushButton("Recommend")
        recommend_button.clicked.connect(self.recommend_movies)
        layout.addWidget(recommend_button)

        self.result_textedit = QTextEdit()
        self.result_textedit.setReadOnly(True)
        layout.addWidget(self.result_textedit)

        self.setCentralWidget(central_widget)

    def recommend_movies(self):
        genre = self.genre_combo.currentText()
        cf = self.cf_slider.value() / 100.0
        rating = self.rating_slider.value() / 10.0
        language = self.language_edit.text()
        release_year_min = self.release_year_spinbox_min.value()
        release_year_max = self.release_year_spinbox_max.value()
        runtime_min = self.runtime_spinbox_min.value()
        runtime_max = self.runtime_spinbox_max.value()

        release_year_range = list(range(release_year_min, release_year_max + 1))
        runtime_range = f"{runtime_min} {runtime_max}"

        recommender = MovieRecommender()
        recommender.reset()
        recommender.declare(
            UserPreference(
                genre=genre,
                cf=cf,
                rating=rating,
                language=language,
                release_year_range=release_year_range,
                runtime_range=runtime_range,
            )
        )
        recommender.recommend()
        recommendations = recommender.recommendations

        if recommendations:
            result_text = "Recommended movies:\n"
            for rec in recommendations:
                result_text += f"- {rec[0]}\n"
        else:
            result_text = "No movies found matching your preferences."

        self.result_textedit.setPlainText(result_text)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MovieRecommenderApp()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

c:\Users\mudar\anaconda3\envs\Tenserflow\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
